# Tutorial 1 - Understand YOLO Format
#### This is the first tutorial of a series step-by-step walkthroughs of YOLO algorithm.  

In this tutorial, we are going to focus on **YOLO2**, which is simpler to understand and easier to explain comparing to its sucessors. If you are looking for YOLO3/4, I still encourage you to understand YOLO2 first as later modifications are **very similar** to YOLO2 in terms of detection logics.


In [1]:
import tensorflow.keras as tfk
import numpy as np
# please download the h5 file first 
MODEL_PATH = 'yolov2_voc.h5'
model = tfk.models.load_model(MODEL_PATH)

### 1. YOLOv2 architecture
To understand how YOLO works, it is critical to have a general idea of what YOLO looks like. ![architecture](Misc/YoloArchitecture.jpg)

YOLO is built on top of a VGG-style classification network called Darknet, which serves as a feature extractor(Sort of like transfer learning). With that being said, there are many different types of YOLOs that built on top of variants of darknets. Here is a list of YOLO architectures:
* [yolo tiny](https://github.com/pjreddie/darknet/blob/master/cfg/yolov2-tiny-voc.cfg) 
* [yolo2](https://github.com/pjreddie/darknet/blob/master/cfg/yolov2-voc.cfg)
* [yolo3](https://github.com/pjreddie/darknet/blob/master/cfg/yolov3-voc.cfg)
* ...  

Unlike classifiers that output a one dimension vector(1XNUM_CLASS), YOLO outputs a 3D *tensor*

In [2]:
randomImgs = np.random.randn(2, 416,416,3)  # some random input, just for demo
pred = model.predict(randomImgs)
print('INPUT IMG SHAPE:{}\tMODEL OUTPUT SHAPE:{}'.format(randomImgs.shape, pred.shape))

INPUT IMG SHAPE:(2, 416, 416, 3)	MODEL OUTPUT SHAPE:(2, 13, 13, 125)


As you can see, yolo's output is nothing like what we've seen before. There are 416 x 416 pixels in this image but the output is 13 x 13. How on earth do we interprate the results?

### 2. YOLO format
To localize objects, one navie approach would be predicting the shape of the box and the object class prob. for every pixel. For an image with shape 416 x 416 x 3(RGB), the output would be like 416 x 416 x (2+CLASS_NUM), where the last dimension could be something like this: (box width, box height, soft maxed class probabilities)
![](Misc/Naive.jpg)
<center>A navie approache - one box per pixel</center>

Yolo is engineered to be fast and accurate. Therefore, it is not ideal to predict one box per pixel (two adjecent pixel may belong to one object). The geeks who invented YOLO went to work and came up with a better idea.

##### Optimization 1 - reduce predicted boxes
Instead of predicting one box per pixel, we divide an image into S x S grids and predict several boxes *per grid*.  
For yolo v2, S = image shape / 32 (convolutions shrink the size of feature maps). If the image is 416 x 416, we have 13 x 13 grids.
![Grids](Misc/Grids.jpg)

##### Optimization 2 - object score for filtering out low confidence prediction
Yolo also introduces an **object score** in addition to classification probabilities. The object score is an estimation of whether an object appears in the predicted box (It doesn't care what object, that's the job of class probailities). If a prediction has low object score, it will be discarded in post-processing. With that being said, the bounding boxe should like like this ![box](Misc/Better.jpg)
<center>Optimized bounding box prediction with obj score</center>

##### Optimization 3 - tailor to the dataset
Instead of predicting the size of boxes w.r.t to the entire image, YOLO introduces what is known as **Anchor Box**, a list of predefined shapes that best match the desired objects. YOLO predicts the shape of bounding boxes **w.r.t the anchor boxes**. More precisely, yolo predicts:  
1. offset of the box center (tx and ty in the figure below, need to pass through a sigmoid func) to the top left corner of a grid
2. the width and height of the box w.r.t an anchor box (tw and th in the figure below. pw and ph are the size of the anchor)

The coordinates of a box is (bx, by, bw, bh), all w.r.t to the size of the grid. To convert it to full-image scale, multiply by 32 and offset by the central coordinates of the grid
![BoxPred](Misc/BoxPred.jpg)

The shape of anchor boxes is unique to dataset and is found by running K-mean on ground truth bounding boxes. Here's the predefined anchors for the Pascal dataset used by YOLOV2. 
<pre>
                                                    Widht, Height (w.r.t to grids)
                                                    1.3221, 1.73145
                                                    3.19275, 4.00944
                                                    5.05587, 8.09892
                                                    9.47112, 4.84053
                                                    11.2364, 10.0071
</pre>

![Anchor](Misc/AnchorBoxes.jpg)
<center>Several anchor boxes per grid</center>

As you can see, there are anchor for small objects and large boxes for bigger objects. There are tall, slim boxes for objects like human to wide, short boxes for objects such as bus. The use of anchor box significantly boosts the detection performance of YOLO



### 3. Final Format
Now you know YOLO predicts **several bounding boxes** per grid instead of just one. The output shape would be something like 13 x 13 x NUM_ANCHOR X (BOX INFO), where the last dimension looks just like an upgraded version of the naive approache.
![Upgraded](Misc/Optimized.jpg)

For the Pascal dataset, there are 20 classes and 5 anchor boxes being used. Therefore, the output shape should be:  
<pre>
                                    13 x 13 x 5 x (4 coordinates + obj score + 20 class prob))
                                    = 13 x 13 x 5 x 25
                                    = 13 x 13 x 125 (flattend)
</pre>

In this tutorial, I explained the basic idea of the yolo algorithm. In the next tutorial, we are going to convert the raw output to a list of boxes that can be displayed on image scales. 